# Álgebra Linear para ML

Observação, se você esta achando que eu me dei o trabalho de escrever todos essas formulas em $LaTeX$, pense de novo, talvez isso te ajude a transformar imagens em LaTeX ... https://p2t.breezedeus.com, o modelo é gratuito e da pra baixar usando o HuggingFace.

**Nome: Eduardo D. Luna**

## Imports

In [1]:
import numpy as np
import scipy.linalg as la
from scipy.linalg import svd, lu_factor, lu_solve

## Exercício 1


Fiz por eliminação de Gauss o posto, mas fiquei com um pouco de preguiça de resolver na mão o sistema. Então aqui vamos um compilado de uso das tecnicas de resolução de sistema.

Curiosidades, pelo visto o `numpy` para o `solve` usa LU, para `lstsq` usa o QR, e quando a matriz não tem o posto correto no `lstsq` ele usa o SVD.  

In [2]:
A1 = np.array([
    [2, 1, 1],
    [4, -6, 0],
    [-2, 7, 2]
    ])

b1 = np.array([5, -2, 9])

Ab1 = np.column_stack((A1, b1))

Ab1[2] = Ab1[0] + Ab1[2]
Ab1[1] = Ab1[1] - 2*Ab1[0]
Ab1[2] = Ab1[1] + Ab1[2]

posto = np.count_nonzero(np.any(Ab1 != 0, axis=1))

solucao_qr = np.linalg.lstsq(A1, b1, rcond=None)[0]
solucao_lu = np.linalg.solve(A1, b1)
solucao_plu = lu_solve(lu_factor(A1), b1)
solucao_inv = np.linalg.inv(A1) @ b1
svd_np = np.linalg.svd(A1, compute_uv=True, full_matrices=True)
svd_sp = svd(A1, compute_uv=True, full_matrices=True)

print(f"Posto da matriz: {posto} e esta correto? {posto == np.linalg.matrix_rank(A1)}")


print(f"SVD iguais? {np.allclose(svd_np[0], svd_sp[0]) and np.allclose(svd_np[1], svd_sp[1]) and np.allclose(svd_np[2], svd_sp[2])}")

U, sigma, V = svd_np

c = U.T @ b1
w = np.diag(1/sigma) @ c
solucao_svd = V.conj().T @ w


print(f"Solução com QR: {solucao_qr}")
print(f"Solução com LU: {solucao_lu}")
print(f"Solução com PLU: {solucao_plu}")
print(f"Solução com SVD: {solucao_svd}")
print(f"Solução com Inversa: {solucao_inv}")

Posto da matriz: 3 e esta correto? True
SVD iguais? True
Solução com QR: [1. 1. 2.]
Solução com LU: [1. 1. 2.]
Solução com PLU: [1. 1. 2.]
Solução com SVD: [1. 1. 2.]
Solução com Inversa: [1. 1. 2.]


In [3]:
A2 = np.array([
    [1 , 2, 1],
    [-1, 0, 3],
    [1 ,-2, 1]
    ])

b2 = np.array([0,5,1])

Ab2 = np.column_stack((A2, b2))

Ab2[2] = Ab2[2] - Ab2[0]
Ab2[1] = Ab2[1] + Ab2[0]
Ab2[2] = 2*Ab2[1] + Ab2[2]

posto = np.count_nonzero(np.any(Ab2 != 0, axis=1))

solucao_qr = np.linalg.lstsq(A2, b2, rcond=None)[0]
solucao_lu = np.linalg.solve(A2, b2)
solucao_plu = lu_solve(lu_factor(A2), b2)
solucao_inv = np.linalg.inv(A2) @ b2
svd_np = np.linalg.svd(A2, compute_uv=True, full_matrices=True)
svd_sp = svd(A2, compute_uv=True, full_matrices=True)

print(f"Posto da matriz: {posto} e esta correto? {posto == np.linalg.matrix_rank(A1)}")


print(f"SVD iguais? {np.allclose(svd_np[0], svd_sp[0]) and np.allclose(svd_np[1], svd_sp[1]) and np.allclose(svd_np[2], svd_sp[2])}")

U, sigma, V = svd_np

c = U.T @ b2
w = np.diag(1/sigma) @ c
solucao_svd = V.conj().T @ w


print(f"Solução com QR: {solucao_qr}")
print(f"Solução com LU: {solucao_lu}")
print(f"Solução com SVD: {solucao_svd}")
print(f"Solução com Inversa: {solucao_inv}")

Posto da matriz: 3 e esta correto? True
SVD iguais? True
Solução com QR: [-0.875 -0.25   1.375]
Solução com LU: [-0.875 -0.25   1.375]
Solução com SVD: [-0.875 -0.25   1.375]
Solução com Inversa: [-0.875 -0.25   1.375]


## Exercício 2

Para provar que algo é um espaço vetorial, tem que seguir 8 condições:

 $$
 \begin{split}
 (1) & \quad (\vec{u} + \vec{v}) + \vec{w} = \vec{u} + (\vec{v} + \vec{w}) \\
 (2) & \quad \vec{0} + \vec{u} = \vec{u} + \vec{0} = \vec{u} \\
 (3) & \quad \vec{u} + (-\vec{u}) = \vec{0} \\
 (4) & \quad \vec{u} + \vec{v} = \vec{v} + \vec{u} \\
 (5) & \quad \alpha (\vec{u} + \vec{v}) = \alpha \vec{u} + \alpha \vec{v} \\
 (6) & \quad (\alpha + \beta)\vec{v} = \alpha\vec{v} + \beta\vec{v} \\
 (7) & \quad (\alpha\beta)\vec{v} = \alpha (\beta\vec{v}) \\
 (8) & \quad 1\vec{u} = \vec{u}
 \end{split}
 $$


### a) $M(m,n)$ é um espaço Vetorial

Matrizes são fechados para soma, então as primeiras 4 estão ok. O 5, 6 e o 8, tambem, já o 7, você pode pensar como reduzir isso a um único escalar e depois executar uma multiplicação por escalar. 

### b) $P_n$ é um espaço vetorial:

$$
 P_n = \{ \; p(x) = a_0 + a_1 x + ... a_n + x^n \; \mid \; a_i \in \mathbb{R} \; \}
$$

Para simplificar eu vou escever assim:

$$
P_n = \{ \; p(x) = \sum_{i}^n a_i x^n \; \}
$$


Mesma coisa, polinomios de grau menor que $n$ são fechados para soma então os 4 primeiros estão ok. Para o 5,6,7 e 8:

$$
\forall \alpha \in \mathbb{R} \; \; \alpha \sum_{i}^n a_i x^n \; \text{é valido}
$$

## Exercício 3

Se $U \subseteq W$, então $U \cup W = W$, que é subespaço.
Se $W \subseteq U$, então $U \cup W = U$, que é subespaço.

Suponha que $U \cup W$ é subespaço e que $U \nsubseteq W$ e $W \nsubseteq U$.

Então existem $u \in U$ com $u \notin W$ e $w \in W$ com $w \notin U$.

Como $U \cup W$ é subespaço, $u + w \in U \cup W$.

Logo, $u + w \in U$ ou $u + w \in W$.

- Se $u + w \in U$, então $w = (u + w) - u \in U$ (pois $U$ é subespaço). Contradição!
- Se $u + w \in W$, então $u = (u + w) - w \in W$ (pois $W$ é subespaço). Contradição!

Portanto, $U \subseteq W$ ou $W \subseteq U$.

## Exercício 4

Seja $F(X,V) = \{ f : X \to V \}$ onde V é espaço vetorial.


Definimos:
- $(f + g)(x) = f(x) + g(x)\quad \forall\, x \in X$
- $(\alpha f)(x) = \alpha \cdot f(x)\quad \forall\, x \in X,\; \alpha \in \mathbb{R}$

Pela definição acima todos os axiomas são validos. Por tanto é espaço vetorial.

## Exercício 5

In [4]:
# Definindo as matrizes
M1 = np.array([[1, 1], [1, 1]])
M2 = np.array([[1, -1], [-1, 1]])
M3 = np.array([[-1, 2], [2, -1]])

# Vetorizando as matrizes (flatten)
v1 = M1.flatten()
v2 = M2.flatten()
v3 = M3.flatten()

# Criando matriz com vetores como colunas
A = np.column_stack([v1, v2, v3])
print("\nMatriz com vetores como colunas:")
print(A)

# Calculando o posto
posto = np.linalg.matrix_rank(A)
print(f"\nPosto da matriz: {posto}")
print(f"Número de vetores: {A.shape[1]}")

if posto == A.shape[1]:
    print("\n As matrizes são LINEARMENTE INDEPENDENTES")
else:
    print("\n As matrizes são LINEARMENTE DEPENDENTES")



Matriz com vetores como colunas:
[[ 1  1 -1]
 [ 1 -1  2]
 [ 1 -1  2]
 [ 1  1 -1]]

Posto da matriz: 2
Número de vetores: 3

 As matrizes são LINEARMENTE DEPENDENTES


## Exercício 6


 Seja $\Phi: V \to W$ uma transformação linear.
 
 Ker($\Phi$) é subespaço de V:
 
 $$\mathrm{Ker}(\Phi) = \left\{ v \in V \mid \Phi(v) = 0_W \right\}$$
 
 1.  $\Phi(0_V) = 0_W$ 
 2.  Se $u, v \in \mathrm{Ker}(\Phi)$, então $\Phi(u + v) = \Phi(u) + \Phi(v) = 0_W + 0_W = 0_W$ 
 3.  Se $v \in \mathrm{Ker}(\Phi)$, $\alpha \in \mathbb{R}$, então $\Phi(\alpha v) = \alpha\Phi(v) = \alpha \cdot 0_W = 0_W$ 

 
Im($\Phi$) é subespaço de W:
 
 $$\mathrm{Im}(\Phi) = \left\{ w \in W \mid \exists v \in V,\ \Phi(v) = w \right\}$$
 
 1.  $\Phi(0_V) = 0_W$ 
 2.  Se $w_1, w_2 \in \mathrm{Im}(\Phi)$, existem $v_1, v_2$ com $\Phi(v_1) = w_1$, $\Phi(v_2) = w_2$.

Logo, $w_1 + w_2 = \Phi(v_1) + \Phi(v_2) = \Phi(v_1 + v_2) \in \mathrm{Im}(\Phi)$

 3.  Se $w \in \mathrm{Im}(\Phi)$, existe $v$ com $\Phi(v) = w$.

Logo, $\alpha w = \alpha \Phi(v) = \Phi(\alpha v) \in \mathrm{Im}(\Phi)$ 

Portanto, **Ker($\Phi$) é subespaço de V** e **Im($\Phi$) é subespaço de W**.

## Exercício 7

In [5]:
b = np.array([1, 2, 2])
x = np.array([1, 1, 1])

((x @ b) / np.linalg.norm(b)**2) * b


array([0.55555556, 1.11111111, 1.11111111])

## Exercício 8

In [6]:
A = np.array([[1, 1], [1, 1.0001]])
B = np.array([[0.0001, 1], [1, 1]])


# Calculando números de condicionamento (usando norma 2)
cond_A = np.linalg.cond(A, 2)
cond_B = np.linalg.cond(B, 2)

print(f"\nNúmero de condicionamento de A (norma 2): {cond_A:.2e}")
print(f"Número de condicionamento de B (norma 2): {cond_B:.2e}")


# Sistema 1: A*[x,y]^T = [2, 2]^T
b1 = np.array([2, 2])
sol_A1 = np.linalg.solve(A, b1)

print("\nSistema 1: A*[x,y]^T = [2, 2]^T")
print(f"Solução: x = {sol_A1[0]:.6f}, y = {sol_A1[1]:.6f}")

# Sistema 2: A*[x,y]^T = [2, 2.0001]^T (pequena perturbação)
b2 = np.array([2, 2.0001])
sol_A2 = np.linalg.solve(A, b2)

print("\nSistema 2: A*[x,y]^T = [2, 2.0001]^T")
print(f"Solução: x = {sol_A2[0]:.6f}, y = {sol_A2[1]:.6f}")


Número de condicionamento de A (norma 2): 4.00e+04
Número de condicionamento de B (norma 2): 2.62e+00

Sistema 1: A*[x,y]^T = [2, 2]^T
Solução: x = 2.000000, y = 0.000000

Sistema 2: A*[x,y]^T = [2, 2.0001]^T
Solução: x = 1.000000, y = 1.000000


## Exercício 9

In [7]:

A = np.array([[-3, 1, -3],
              [20, 3, 10],
              [2, -2, 4]])


# Calculando autovalores e autovetores
eigenvalues, eigenvectors = np.linalg.eig(A)

eigenvalues, eigenvectors

(array([-2.+0.00000000e+00j,  3.+1.18630976e-07j,  3.-1.18630976e-07j]),
 array([[ 4.08248290e-01+0.00000000e+00j,  4.47213595e-01-2.19986315e-24j,
          4.47213595e-01+2.19986315e-24j],
        [-8.16496581e-01+0.00000000e+00j,  5.33056353e-16+5.30533855e-08j,
          5.33056353e-16-5.30533855e-08j],
        [-4.08248290e-01+0.00000000e+00j, -8.94427191e-01+0.00000000e+00j,
         -8.94427191e-01-0.00000000e+00j]]))

## Exercício 10

In [8]:

A = np.array([[2, 2],
              [-1, 1]])


U, sigma, V = np.linalg.svd(A)

U,sigma, V

(array([[-1.00000000e+00,  1.11022302e-16],
        [ 8.64164897e-17,  1.00000000e+00]]),
 array([2.82842712, 1.41421356]),
 array([[-0.70710678, -0.70710678],
        [-0.70710678,  0.70710678]]))